# 该程序是用来将Vaihingen数据集的vis_png转换为masks_png的

In [4]:
import numpy as np
from PIL import Image
import glob
import time
import torch
from matplotlib import pyplot as plt 
from tqdm import tqdm, trange

In [5]:
#转换数据集
"""因为，可以看数据集发现，masks_png都是0-1的小数，没有办法训练，所以这里需要转换一下数据集"""
"""给的代码示例是一次转换一张，现在试试一次性全部转换"""
#类别数 背景-0、建筑-1、农田-2、森林-3、草地-4、水-5。

GID_COLOR_MAP = dict(
    Background=(255, 0, 0),#0-背景-红色
    Building=(0, 0, 255), #1-建筑-蓝色
    di_ai_zhi_bei=(0, 255, 255), #2-低矮植被-青色
    Tree=(0, 255, 0), #3-树-绿色
    car=(255, 255, 0),#4-汽车-黄色
    bu_tou_shui_mian=(255, 255, 255), #5-不透水面-白色
)
    
GID_COLORMAP = list(GID_COLOR_MAP.values())
GID_CLASSES = list(GID_COLOR_MAP.keys())

In [6]:
#############用列表来存一个RGB和一个类别的对应################
def GID_colormap2label(GID_COLORMAP):   
    colormap2label = torch.zeros(256**3,dtype = torch.long)
    for i, colormap in enumerate(GID_COLORMAP):
        colormap2label[(colormap[0]*256+colormap[1])*256+colormap[2]] = i
    return colormap2label

#############给定那个列表，和vis_png然后生成masks_png################
def GID_label_indices(vis_png, colormap2label):
    colormap = vis_png.astype('int32')
    idx = (colormap[:,:,0]*256+colormap[:,:,1])*256+colormap[:,:,2]
    return colormap2label[idx]

colormap2label = GID_colormap2label(GID_COLORMAP)  #生成标签的那个查找表列表

In [7]:
#把masks_png转换成vis_png
def render(mask_path, vis_path):
    new_mask = np.array(Image.open(mask_path)).astype(np.uint8) #把原来的图片读出来，存成uint8的格式
    cm = np.array(GID_COLORMAP).astype(np.uint8) #取出COLOR_MAP中的值，存到列表
    color_img = cm[new_mask]  #这里就变成1024*1024*3的了？ why？？
    color_img = Image.fromarray(np.uint8(color_img)) #从np的array格式转换成PIL格式
    color_img.save(vis_path)

#把vis_png转换成masks_png
def create_masks_png(vis_path, masks_path):
    vis_img = np.array(Image.open(vis_path)) #打开vis_png
    masks_png = GID_label_indices(vis_img,colormap2label).numpy()
    color_img = Image.fromarray(np.uint8(masks_png)) #从np的array格式转换成PIL格式
    color_img.save(masks_path)

In [10]:
#数据集的地址    
vis_file_path = 'D:/ATL/Graduation_design/data/Potsdam/Potsdam_data_6000/Val/labels_RGB'
masks_path = 'D:/ATL/Graduation_design/data/Potsdam/Potsdam_data_6000/Val/labels'    #要保存的地方

#数据集的地址列表
# vis_list = np.array(sorted(glob.glob(vis_file_path + '*.tif'))) #把这个文件夹中的图片的地址，以有序的列表形式存放    
vis_list = os.listdir(vis_file_path)

if not os.path.exists(masks_path):
    os.mkdir(masks_path)

In [11]:
if __name__ == '__main__':
    start = time.time()
    for index in trange(len(vis_list), desc='vis_png 转换为 masks_png ing......'):
        create_masks_png(os.path.join(vis_file_path,vis_list[index]),os.path.join(masks_path,vis_list[index]))
    end = time.time()
    print(f'共转换vis_png {len(vis_list)} 张,耗时 {(end-start)} 秒')

vis_png 转换为 masks_png ing......: 100%|██████████| 13/13 [00:11<00:00,  1.12it/s]

共转换vis_png 13 张,耗时 11.570314645767212 秒
